In [1]:
using Symbolics
using NLsolve
using LinearAlgebra


In [2]:
function solvesystem(α_val, p_val, he_val, hp_val, q_val, initguess)
    function system!(F, x)
        uu, ud, du, dd = x
        
        F[1] = uu*(1 - α_val*p_val/2 - (1-α_val)*(1-p_val)*(1-he_val)*(1-uu-ud)^q_val) +
               ud*α_val*(p_val/2 + (1-p_val)*hp_val + (1-p_val)*(1-hp_val)*(uu+ud)^q_val) +
               du*(1-α_val)*(1 - (1-p_val)*(1-he_val)*(1-uu-ud)^q_val) - uu

        F[2] = uu*α_val*p_val/2 +
               ud*((1-α_val)*(1-p_val)*(he_val + (1-he_val)*(uu+ud)^q_val) + α_val*(1-p_val/2) - α_val*(1-p_val)*(hp_val + (1-hp_val)*(uu+ud)^q_val)) +
               dd*(1-α_val)*(1-p_val)*(he_val + (1-he_val)*(uu+ud)^q_val) - ud

        F[3] = uu*(1-α_val)*(1-p_val)*(1-he_val)*(1-uu-ud)^q_val +
               du*((1-α_val)*(1-p_val)*(1-he_val)*(1-uu-ud)^q_val + α_val*(1 - p_val/2 - (1-p_val)*hp_val - (1-p_val)*(1-hp_val)*(1-uu-ud)^q_val)) +
               dd*α_val*(p_val/2 + (1-p_val)*hp_val) - du

        F[4] = uu + ud + du + dd - 1
    end

    sol = nlsolve(system!, initguess)
    return sol.zero

end

solvesystem (generic function with 1 method)

In [3]:
function checkstability(alpha_, p_, he_, hp_, q_, initialguess)
@variables uu ud du dd α p he hp q


    # Define the symbolic system equations
    F1 = uu*(1 - α*p/2 - (1-α)*(1-p)*(1-he)*(1-uu-ud)^q) +
        ud*α*(p/2 + (1-p)*hp + (1-p)*(1-hp)*(uu+ud)^q) +
        du*(1-α)*(1 - (1-p)*(1-he)*(1-uu-ud)^q) - uu

    F2 = uu*α*p/2 +
        ud*((1-α)*(1-p)*(he + (1-he)*(uu+ud)^q) + α*(1-p/2) - α*(1-p)*(hp + (1-hp)*(uu+ud)^q)) +
        dd*(1-α)*(1-p)*(he + (1-he)*(uu+ud)^q) - ud

    F3 = uu*(1-α)*(1-p)*(1-he)*(1-uu-ud)^q +
        du*((1-α)*(1-p)*(1-he)*(1-uu-ud)^q + α*(1 - p/2 - (1-p)*hp - (1-p)*(1-hp)*(1-uu-ud)^q)) +
        dd*α*(p/2 + (1-p)*hp) - du

    F4 = 1 - (F1 + F2 + F3)

    # Create the system vector and variable vector
    F_vec = [F1, F2, F3, F4]
    vars = [uu, ud, du, dd]

    # Calculate the symbolic Jacobian
    # J_symbolic = Symbolics.jacobian(F_vec, vars)


    J = Symbolics.jacobian(F_vec, vars)

    # === Optional: simplify the Jacobian ===
    J_simplified = simplify.(J)

    # === Display results ===
    # println("\nSymbolic 4×4 Jacobian matrix J(uu, ud, du, dd):\n")
    # display(J_simplified)

    # === Example: evaluate numerically ===
    subs_dict = Dict(α => 0.1, p => 0.1, he => 0.01, hp => 0.0, q => 3.0,
                    uu => 0.94, ud => 0.05, du => 0.001, dd => 0.009)

    J_num = substitute(J_simplified, subs_dict) |> eval
    # println("\nNumeric Jacobian at example point:\n")
    # display(J_num)



    solution_point = solvesystem(alpha_, p_, he_, hp_, q_, initialguess)
    println("Fixed point solution: ", solution_point)
    # println("Solution: ", [solution_point[1]+solution_point[2], solution_point[1]+solution_point[3]])


    # Calculate eigenvalues
    eigenvalues = eigvals(J_num)

    # # println("Jacobian matrix:")
    # # display(J_matrix)
    println("Eigenvalues: ", eigenvalues)
    # println("Real parts of eigenvalues: ", real.(eigenvalues))
    # println("Maximum real part: ", maximum(real.(eigenvalues)))

    # Stability analysis
    if all(abs.(eigenvalues) .< 1)
        println("Fixed point is STABLE")
        return true
    else
        println("Fixed point is UNSTABLE")
        return false
    end
    
end

checkstability (generic function with 1 method)

In [4]:
checkstability(0.1, 0.1, 0.01, 0.0, 3, [0., 0., 0.1, 0.9])

Fixed point solution: [0.009529301478604332, 0.008461497498668554, 0.05150419885269053, 0.9305050021700366]
Eigenvalues: [-0.9104708325709805, -0.7769284702325447, -0.08058955319647337, -5.551115123125783e-17]
Fixed point is STABLE


true

In [5]:
checkstability(0.1, 0.1, 0.01, 0.0, 3, [1, 0., 0., 0.])

Fixed point solution: [0.9430601958488656, 0.050236522591181304, 3.7081536233784325e-5, 0.006666200023719509]
Eigenvalues: [-0.9104708325709805, -0.7769284702325447, -0.08058955319647337, -5.551115123125783e-17]
Fixed point is STABLE


true

In [15]:
checkstability(0.1, 0.2375, 0.01, 0.0, 3, [0., 0., 0., 1.])

Fixed point solution: [0.11415086909629747, 0.008280469246148777, 0.12174206983666933, 0.7558265918208847]
Eigenvalues: [-0.9104708325709805, -0.7769284702325447, -0.08058955319647337, -5.551115123125783e-17]
Fixed point is STABLE


true

In [14]:
solvesystem(0.1, 0.2375, 0.01, 0.0, 3, [0., 0., 0., 1.])

4-element Vector{Float64}:
 0.11415086909629747
 0.008280469246148777
 0.12174206983666933
 0.7558265918208847